In [80]:
import torch
import torch.nn.functional as F

In [2]:
words = open('names.txt','r').read().splitlines()

In [3]:
words[:3]

['emma', 'olivia', 'ava']

In [43]:
chars = ['.'] + sorted(set(''.join(words)))
stoi = {i:j for j,i in enumerate(chars)}
itos = {i:j for i,j in enumerate(chars)}

In [21]:
N = torch.zeros((27,27), dtype = torch.int32)

In [41]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1,ix2] +=1

In [45]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [46]:
P = (N+1).float()
P /= P.sum(axis = 1, keepdims = True)

In [57]:
# P[:,0].sum() #column sum

tensor(3.0023)

In [64]:
#for i in range(5):
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    ix = 0
    out = []
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        if ix == 0:
            break
        ch = itos[ix]
        out.append(ch)
    print(''.join(out))

mor
axx
minaymoryles
kondlaisah
anchshizarie


In [74]:
#evaluation
log_likelihood = 0.0
n = 0
for w in 'asheesh':
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        p = P[ix1,ix2]
        logp = torch.log(p)
        log_likelihood += logp
        n +=1
print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'normalized nll: {nll/n}')

log_likelihood=tensor(-31.8496)
nll=tensor(31.8496)
normalized nll: 2.2749736309051514


In [79]:
#create dataset
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print(num)

228146


In [99]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27),generator = g, requires_grad = True)

for i in range(50):
    #forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1,  keepdims = True)
    ypred = probs[torch.arange(num),ys]
    loss = -ypred.log().mean()
    print(loss.item())
    #backward pass
    W.grad = None
    loss.backward()

    #update 
    W.data += -50*W.grad 

3.758953332901001
3.371100664138794
3.154043197631836
3.020373582839966
2.927711248397827
2.8604023456573486
2.8097290992736816
2.7701022624969482
2.7380728721618652
2.711496353149414
2.6890029907226562
2.6696884632110596
2.6529300212860107
2.638277292251587
2.6253881454467773
2.613990545272827
2.60386323928833
2.5948216915130615
2.5867116451263428
2.579403877258301
2.572789192199707
2.5667762756347656
2.5612878799438477
2.5562589168548584
2.551633834838867
2.547366142272949
2.543415069580078
2.5397486686706543
2.5363364219665527
2.5331544876098633
2.5301806926727295
2.5273969173431396
2.5247862339019775
2.522334575653076
2.520028829574585
2.517857789993286
2.515810966491699
2.513878345489502
2.512052059173584
2.510324001312256
2.5086867809295654
2.5071346759796143
2.5056614875793457
2.5042612552642822
2.502929210662842
2.5016613006591797
2.5004520416259766
2.4992988109588623
2.498197317123413
2.497144937515259


In [100]:
#sampling from NN
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0

    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W
        counts = logits.exp()
        probs = counts / counts.sum(1,  keepdims = True)
        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        if ix == 0:
            break
        out.append(itos[ix])
    print(''.join(out))

mor
axx
minaymoryles
kondmaisah
anchthizarie
